In [1]:
from typing import Union
import header
from header import  pprint, start_supplier, Product, ProductFields, Supplier, StringFormatter, StringNormalizer

supplier_prefix = 'amazon'
s = start_supplier(supplier_prefix)


""" s - на протяжении всего кода означает класс `Supplier` """



->INFO 19:07:59 from start: 0:00:00.006524 User {{user_id}} logged in
	helpers.logger:<module>:48



NoneType: None


->INFO 19:07:00 from start: 0:00:00.652396 caller: __init__ file: <attrs generated init gs.GlobalSettings>
	gs:__init__:72

True
->INFO 19:07:00 from start: 0:00:01.132260 amazon begin to attach driver ...
	suppliers.supplier:_load_settings:126

->INFO 19:07:00 from start: 0:00:01.133260  
        ----------------------
        Старт вебдрайвера firefox. 
        Будет выбран драйвер, подключатся настройки и профиль.
        На моем компьютере это занимает около полуминуты
        Ждем...
        ----------------------
        
	webdriver.Driver:__init__:206

->INFO 19:07:00 from start: 0:00:01.134260  Firefox ... 
	webdriver.Firefox:__init__:71


            ->DEBUG 19:07:06 from start: 0:00:07.029941 Профиль C:\Users\user\AppData\Local\Temp\tmpdl5d1150\webdriver-py-profilecopy\user.js
            	webdriver.Firefox:__init__:89

->INFO 19:07:41 from start: 0:00:42.032476  ... стартовал 
	webdriver.Firefox:__init__:147

->INFO 19:07:41 from start: 0:00:42.034240  ... driver attached su

' s - на протяжении всего кода означает класс `Supplier` '

In [2]:
from header import  pprint, start_supplier, Product, ProductFields, Supplier, StringFormatter, StringNormalizer

def grab_product_page(s: Supplier, id_product: int = 0 , presta_api_version: Union[str('V1'),str('V2'),str('V3')] = 'V3') -> ProductFields:
    """ ПОКА ТОЛЬКО ДЛЯ НОВЫХ!!!!! СТРАНИЦ
        Собираю информацию со страницы товара. 
        Важно помнить, что драйвер уже должен быть на
        этой странице
        ---------------
        Attrs:
            s (Supplier)
        @returns
            f (ProductFields) с заполненными полями, else False
    """


    l = s.reread_locators('product')
    d = s.driver
   
    _ = d.execute_locator

    print('Start  function grabber for ')

    def add_new_product_stage_1(s: Supplier) -> ProductFields:
        """ Первая стадия добавления нового товара. Заполняю все необходимые поля
            Далее я отправлю их в PrestaShop и получу обратно ID вновь созданного товара
            На второй стадии, зная ID, я отправлю главную картинку и прочее
        """
        f: ProductFields = ProductFields()
       
        """ ID,ASIN,SKU,SUPPLIER SKU """
    

        def _set_defaults() -> bool:
            """ Set defaults for product of supplier """
            f.active = 1
            f.on_sale = 1
            f.min_qty = 1
            f.low_stock_level = 0
            f.low_stock_threshold = ''
            f.show_price = 1
            f.show_condition = 1
            f.aviable_online_only = 0
            f.advanced_stock_management = 0
            f.state = 1

        def set_price(s, format: str = 'str') -> Union[str,float]:
            """ Привожу денюшку к 
            [ ] float 
            [v] str
            """
            l = s.reread_locators('product')
            raw_price = _(l['price']['new'])[0]
            raw_price = str(raw_price).split('\n')[0]
            return StringNormalizer.normalize_price(raw_price)
    

        """ Я добавляю в базу данных престашоп товар путем нескольких последовательных действий
        1. Добавляю поля, необходимые для создания нового товара
        2. Получаю `id_product` созданного товара
        3. Используя полученный `id_product` загружаю дефолтную картинку
        4. итд.
        """

        _set_defaults()
        l = s.reread_locators('product')
        ASIN = _(l['ASIN'])
        f.reference = f'{s.supplier_id}-{ASIN}'
        f.supplier_reference = ASIN
        f.price = set_price(s, format = 'str')
        f.name = _(l['name'])[0]
        
        f.description_short = _(l['description_short'])[0]
        f.id_supplier = s.supplier_id
        
        _category_default = list(s.current_scenario['presta_categories']['default_category'].keys())[0]
        f.id_category_default = _category_default
        f.category_ids = Product.get_parent_categories(_category_default)

        affiliate = _(l['affiliate short link'])
        affiliate = affiliate[1][0]
        f.affiliate_short_link = affiliate

        #f.link_rewrite = d.current_url.split(f'''/''')[-4]



        f.link_rewrite = "test-link"
        return f

    def upload_image(id_product, image_url) -> ProductFields:
        """ После того, как я занес новый товар в бд - я получу его id
        Далее я гружу фото и получаю ее id
        Далее я догружаю осталные па
        ----------------------
        Attrs: 
            s (Supplier):
            f (ProductFields): Заполненные на первом этапе поля. Я беру из них только то, что мне надо для апдейта
        @returns
            ProductFields: поля для апдейта
        """

        img = Product.upload_product_default_image(id_product, image_url)
        return img
        
    



    if id_product == 0:
        print("---------------------NEW PRODUCT-----------------------")
        f = add_new_product_stage_1(s)
        product_dict = f.product_dict
        pprint(product_dict)
        product = Product.add_2_PrestaShop(product_dict, presta_api_version)['product']
        pprint(product)
        
        id_product = product['id']
        print("---------------------NEW PRODUCT ID-----------------------")
        print(id_product)
        image_url = _(l['additional_images_urls'])[0]
        img = upload_image(id_product, image_url)
        #return product, img

    
    ...


In [3]:
from header import j_loads, j_dumps
def run_scenarios(s) -> bool:
    """
    Runs the set of scenarios ...ed in scenario_files. If the list of scenarios is not ...ed, it is taken from 
    s.settings['scenarios']. For each scenario in the list, the function run_scenario_file(s, current_scenario_filename)
    is called, which loads the JSON scenario file and ...es it to the function run_scenario(s, scenario).
    
    @param
        - s: An instance of the Supplier class, which stores the settings and parameters needed for executing the scenario.
        - scenario_files: A list of file paths for the JSON scenario files to be executed. If None, the default list of 
                        scenarios from s.settings['scenarios'] will be used.

    @returns
        - A boolean indicating whether all scenarios were executed succesStringFormatterully (True) or not (False).
    """

    
    if not s.scenario_files:
        logger.error(f'''Нет файлов сценариев для исполнения
       Поставщик {s.supplier_prefix}''')
        return

    for current_scenario_filename in s.scenario_files:
        """ Execute each scenario file one by one """

        run_scenario_file(s, current_scenario_filename)
        s.settings['last_runned_scenario'] = current_scenario_filename


    # Return True to indicate succesStringFormatterul execution of all scenarios
    return True

def run_scenario_file(s, current_scenario_file_path) -> bool:
    """ 
    Runs the scenario file specified in `current_scenario_filename`.

    @param
        s (Supplier): The Supplier object.
        current_scenario_filename `str`  :  The name of the scenario file in the scenarios directory.

    @returns
        bool: True if the scenario file was succesStringFormatterully run, False otherwise.
    """

    print(current_scenario_file_path)
    scenario = j_loads(current_scenario_file_path)['scenarios']
    run_scenario(s, scenario)


    #if not s.scenarios:
    #    logger.error(f"File {s.current_scenario_file_path} does not contain any scenarios.")
    #    return
    
    #s.export_file_name = f"{s.current_scenario_filename.split('.')[0]}"
    #s.dir_export_imagesECTORY_FOR_STORE = Path(s.dir_export_imagesECTORY, f"{s.current_scenario_filename.split('.')[0]}")

    #update_categories_in_scenario_file(s, current_scenario_filename)
    """ Compares the scenario file to the actual state in the store's categories """

    #for scenario in s.scenario_files:
    #    if scenario['url'] is None:
    #        # """ Not all scenarios have a URL хуй знает почему """
    #        #TODO: это проблема? почему так?
    #        raise ValueError(f""" В сценарии {k} нет URL. """)
    #    run_scenario(s, scenario)
    #return True

def run_scenario(s, scenario: dict) -> bool:
    """ Executes each scenario in the set.
    It receives a list of product URLs in the category and, using the function grab_product_page(),
    extracts data about each product, writing it to a list of products for the supplier.
    If more than 150 products were obtained, the results are written to a separate file.

    @param
        s: class Supplier instance
        scenario: dictionary containing scenario details

    @returns
        True if succesStringFormatterul, False otherwise
    """

    for name, scenario in scenario.items():
        s.current_scenario:dict = scenario
    
        
    d = s.driver


    def _run(s):
        """ Runner function.

        @param
            s: class Supplier instance
        """
        breakpoint()
        list_products_in_category: list =  s.related_modules.get_list_products_in_category(s)
        """ Получаю линки на товары со страницы категории (url сценария) """


        if not list_products_in_category:
            return

        for url in list_products_in_category:
            d.get_url(url)

            grab_product_page(s)

            #p = Product

            #f: ProductFields = p.grab_product_page(s)
            #check = p.check_if_product_in_presta_db(p.fields.reference)

            #if isinstance(check, int):
            #    """ Если товар в бд PrestaShop вернется id_product, иначе False """

            #    p.fields.id_product = check
            #    logger.warning(f""" Товар {jprint(p.fields)} 
            #    -------------------------------
            #    уже есть в бд PrestaShop.
            #    Пока ничего не делаю
            #    """)
            #    ...
            #else:
            #    p.add_2_PrestaShop(p.)

    def _export_grabbed_products():
        #if len(p) > 0:
        #    export_files(s, p, f"{s.export_file_name}-{s.current_scenario['files_counter']}", ["csv"])
        #    s.current_scenario["products_counter"] += 1
        #    if s.current_scenario["products_counter"] > 150:
        #        s.current_scenario["files_counter"] += 1
        #        s.current_scenario["products_counter"] = 0
        #else:
        #    logger.warning("No products to export")
        #    return
        ...

    #1.
    #if not d.get_url(s.current_scenario["url"]):
    #    return

    #2.
    d.get_url(s.current_scenario["url"])
    _run(s)

    #3.
    _export_grabbed_products()

    return True





In [ ]:
run_scenarios(s)